In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
Transactions_df = pd.read_csv('Transactions.csv')

In [5]:
Transactions_df.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


In [7]:
Customers_df = pd.read_csv('Customers.csv')
Products_df = pd.read_csv('Products.csv')

In [11]:
Customers_df.tail()

,CustomerID,CustomerName,Region,SignupDate
195,C0196,Laura Watts,Europe,2022-06-07
196,C0197,Christina Harvey,Europe,2023-03-21
197,C0198,Rebecca Ray,Europe,2022-02-27
198,C0199,Andrea Jenkins,Europe,2022-12-03
199,C0200,Kelly Cross,Asia,2023-06-11


In [13]:
Transactions_df.columns = Transactions_df.columns.str.strip().str.replace('ï»¿', '')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder

merged_df = transactions_df.merge(customers_df, on="CustomerID", how="left").merge(products_df, on="ProductID", how="left")

# Aggregate customer transaction data
customer_features = merged_df.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'Price': 'mean'
}).reset_index()

# Merge customer profile data
customer_data = customers_df.merge(customer_features, on="CustomerID", how="left").fillna(0)

# Encode categorical features
encoder = OneHotEncoder(sparse=False, drop='first')
encoded_region = encoder.fit_transform(customer_data[['Region']])

# Normalize numerical features
scaler = StandardScaler()
numerical_features = scaler.fit_transform(customer_data[['TotalValue', 'Quantity', 'Price']])

# Combine encoded and numerical features
final_features = pd.concat([
    pd.DataFrame(encoded_region, columns=encoder.get_feature_names_out(['Region'])),
    pd.DataFrame(numerical_features, columns=['TotalValue', 'Quantity', 'Price'])
], axis=1)

# Compute cosine similarity
similarity_matrix = cosine_similarity(final_features)

# Create a DataFrame for similarity
similarity_df = pd.DataFrame(similarity_matrix, index=customer_data['CustomerID'], columns=customer_data['CustomerID'])

# Get top 3 similar customers for the first 20 customers
lookalikes = {}
for customer in customer_data['CustomerID'][:20]:
    similar_customers = similarity_df[customer].sort_values(ascending=False).iloc[1:4]
    lookalikes[customer] = [(sim_cust, round(score, 4)) for sim_cust, score in similar_customers.items()]

# Convert to the required format
lookalike_list = [{'cust_id': cust, 'lookalikes': lookalikes[cust]} for cust in lookalikes]
lookalike_df = pd.DataFrame(lookalike_list)

# Save as Lookalike.csv
lookalike_df.to_csv('Lookalike.csv', index=False)

print("Lookalike model results saved to 'Lookalike.csv'.")


In [33]:
merged_df = Transactions_df.merge(Customers_df, on="CustomerID", how="left").merge(Products_df, on="ProductID", how="left")

In [35]:
merged_df['Price'] = merged_df['Price_x'].combine_first(merged_df['Price_y'])

In [37]:
merged_df = merged_df.drop(columns=['Price_x', 'Price_y'])
merged_df.head()


,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,CustomerName,Region,SignupDate,ProductName,Category,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68


In [39]:
customer_features = merged_df.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'Price': 'mean'
}).reset_index()

In [41]:
customer_data = Customers_df.merge(customer_features, on="CustomerID", how="left").fillna(0)

In [43]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_region = encoder.fit_transform(customer_data[['Region']])

scaler = StandardScaler()
numerical_features = scaler.fit_transform(customer_data[['TotalValue', 'Quantity', 'Price']])

In [45]:
final_features = pd.concat([
    pd.DataFrame(encoded_region, columns=encoder.get_feature_names_out(['Region'])),
    pd.DataFrame(numerical_features, columns=['TotalValue', 'Quantity', 'Price'])
], axis=1)

In [47]:
similarity_matrix = cosine_similarity(final_features)

In [49]:
similarity_df = pd.DataFrame(similarity_matrix, index=customer_data['CustomerID'], columns=customer_data['CustomerID'])

In [51]:
lookalikes = {}
for customer in customer_data['CustomerID'][:20]:
    similar_customers = similarity_df[customer].sort_values(ascending=False).iloc[1:4]
    lookalikes[customer] = [(sim_cust, round(score, 4)) for sim_cust, score in similar_customers.items()]

In [53]:
lookalike_list = [{'cust_id': cust, 'lookalikes': lookalikes[cust]} for cust in lookalikes]
lookalike_df = pd.DataFrame(lookalike_list)

In [55]:
lookalike_df.to_csv('Lookalike.csv', index=False)

print("Lookalike model results saved to 'Lookalike.csv'.")

Lookalike model results saved to 'Lookalike.csv'.
